# 3D Image Visualization with PyVista

Installation: `pip install pyvista`

In [2]:
import pyvista as pv
from pyvista import examples

## Volume rendering

Complete example: [Volume Rendering (PyVista)](https://docs.pyvista.org/examples/02-plot/volume#sphx-glr-examples-02-plot-volume-py)

In [ ]:
knee = examples.download_knee_full()

plottter = pv.Plotter()
plottter.add_volume(knee, opacity="sigmoid", cmap="coolwarm", shade=True,)
plottter.show()

## Surface rendering

Complete example: [Physically Based Rendering (PyVista)](https://docs.pyvista.org/examples/02-plot/pbr.html)

In [ ]:
mesh = examples.download_nefertiti()

plotter = pv.Plotter()
plotter.add_mesh(mesh, color='linen', pbr=True, metallic=0.8, roughness=0.1, diffuse=1,)

# Environment background
cubemap = examples.download_sky_box_cube_map()
plotter.add_actor(cubemap.to_skybox())
plotter.set_environment_texture(cubemap)

plotter.show()

## Numpy to PyVista

In [ ]:
from skimage.data import cells3d

image = cells3d()[:, 1]  # A 3D Numpy array

image.shape

In [ ]:
import numpy as np

def to_pyvista(image: np.ndarray) -> pv.ImageData:
    """Convert a 3D numpy array to PyVista volume for rendering"""
    volume = pv.ImageData(dimensions=np.array(image.shape) + 1)
    volume.cell_data["values"] = image.flatten(order="F")
    return volume

vol = to_pyvista(image)

vol.plot(volume=True)